In [1]:
import os
import sys
import time
import napari
import pickle
import h5py
import importlib
import numpy as np
import pandas as pd

from napari import Viewer

from skimage import measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import white_tophat, disk

from napari.qt.threading import thread_worker
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

from tifffile import imread,imsave,TiffFile
import matplotlib.pyplot as plt
import seaborn as sn

from nd2reader import ND2Reader
import btrack

from magicgui import magicgui

sys.path.append(r'D:\imPy\libraries') 
sys.path.append(r'D:\BARC\napari_tracking_manual')

gallery_functions = importlib.import_module('gallery_functions')
my_napari = importlib.import_module('napari_display_functions')
fov_f = importlib.import_module('fovRingsLibrary')

In [2]:
importlib.reload(gallery_functions)
importlib.reload(my_napari)
importlib.reload(fov_f)

<module 'fovRingsLibrary' from 'D:\\imPy\\libraries\\fovRingsLibrary.py'>

In [3]:
myFov = '01'

myDirIm = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tiffs'

# specify tracking channel
myFile_track_im = f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_01.tif'
track_intensity = 'intensity_01_nuc_corr' # name of the column

# specify additional channels
myFile_signal_im = [f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_02.tif',
                    f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_03.tif',
                   f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_04.tif']

colors_list = ['red','green','magenta']
names_list = ['DHB','cyclinD','p21']

# specify columns to plot
to_plot_signals = ['DHB_ratio','intensity_03_nuc_corr','intensity_04_nuc_corr','cyc_D_over_p21'] # name of the column
to_plot_names = ['DHB_ratio [cyt/nuc]','cyclinD','p21','Cyclin D/p21 (mean)']
to_plot_colors = ['red','green','blue','black']


myDirTracks = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
myDirTracks = os.path.join(myDirTracks,myFov)
dfDir = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
dfDir = os.path.join(dfDir,f'{myFov}_napari')

In [4]:
modelPath = r'D:\BARC\Sonja_2021\b_track\cell_config.json'

btrack_init_path = os.path.join(dfDir,f'{myFov}_tracks_init.h5')
btrack_mod_path = os.path.join(dfDir,f'{myFov}_tracks_mod.h5')

## Read in the tracking channel

In [5]:
%%time

# read in image
myIm = imread(os.path.join(myDirIm,myFile_track_im))
myIm.shape

Wall time: 50.8 s


(361, 2765, 2765)

## Read in signal channels

In [6]:
%%time

myIm_signal_list = []
# read in additional signal images
for myFile in myFile_signal_im:
    temp = imread(os.path.join(myDirIm,myFile))
    myIm_signal_list.append(temp)

Wall time: 2min 32s


## Read in tracks in a form of labels

In [7]:
%%time

# read in labels as tracks
myLabels =[]

fovFiles = [x for x in os.listdir(myDirTracks) if f'series_{myFov}_' in x]

for myFile in fovFiles:
    
    # read a mask
    labels = imread(os.path.join(myDirTracks,myFile))
    
    myLabels.append(labels)
    
myLabels = np.array(myLabels)

Wall time: 1min 23s


## Get latest tracking

In [8]:
# check if modified tracking exists

try:
    f = open(btrack_mod_path.replace('h5','pkl'),'rb')
    
    print('Opening corrected tracking.')

except(FileNotFoundError):
  
    f = open(btrack_init_path.replace('h5','pkl'),'rb')
    print('Opening original tracking.')

data, properties, graph = pickle.load(f)

Opening original tracking.


## Get the full data frame

In [9]:
cellDataAll=pd.read_pickle(os.path.join(dfDir,f'cellPose_btrack_regionprops_bck_{myFov}.pkl'))
cellDataAll.drop(['file','x','y'],axis=1,inplace=True)

In [10]:
# create an array for visualization of accepted points
selData=cellDataAll.loc[cellDataAll.accepted==True,:]
acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

In [11]:
cellDataAll.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,intensity_02_nuc_corr,intensity_02_ring_corr,background_03,intensity_03_nuc_corr,intensity_03_ring_corr,background_04,intensity_04_nuc_corr,intensity_04_ring_corr,DHB_ratio,cyc_D_over_p21
0,1,353,7.277620,2400.410765,-1.410806,25.969168,18.036581,0,2388,18,...,1205.090791,227.170204,748.129512,227.595700,33.141729,1934.079928,-41.558682,-31.655091,0.188509,-5.476490
1,2,632,26.928797,1711.330696,-0.934733,30.203461,26.730650,13,1697,42,...,322.688395,388.158446,748.129512,59.300867,29.218314,1934.079928,-27.586257,-14.976667,1.202889,-2.149653
2,3,511,29.324853,2736.291585,-1.315686,38.273719,17.379927,19,2717,39,...,246.690931,27.343267,748.129512,231.623912,52.668360,1934.079928,7.828095,-15.663261,0.110840,29.588795
3,4,555,49.800000,1879.099099,-1.228809,30.717216,23.097163,38,1865,62,...,105.312157,127.468386,748.129512,16.958776,-6.331781,1934.079928,-63.663712,-66.399399,1.210386,-0.266381
4,5,423,56.040189,286.456265,-1.239673,30.591690,17.750233,46,273,66,...,369.661234,66.441352,748.129512,238.402402,39.864340,1934.079928,-22.744231,-18.465174,0.179736,-10.481885


In [ ]:
# other preparations

# create intensity image
intIm = np.zeros([myIm.shape[1],myIm.shape[2],len(myIm_signal_list)+1]).astype('uint16')

## Main viewer

In [12]:
viewer = napari.Viewer()

# tracking layers
viewer.add_image(myIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

track_layer=viewer.add_tracks(data, properties=properties, graph=graph,name='tracking')
track_layer.display_id=True

layer_inProgress = viewer.add_labels(myLabels,name='objects',opacity = 0.5)

# signal layers
for myIm_signal,myColor,myName in zip(myIm_signal_list,colors_list,names_list):
    viewer.add_image(myIm_signal,colormap=myColor,contrast_limits=(0, 4000),opacity = 1,visible=False,name = myName)

# helper layers
layer_accepted = viewer.add_points(acceptedPoints,name='accepted objects',face_color='green', opacity =0.5, ndim=3)
layer_mod = viewer.add_points([],name='modPoints',face_color='red',ndim=3)

In [19]:
# update single object function

@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    global cellDataAll

    regProps = ['label', 'area','centroid','major_axis_length','minor_axis_length','orientation','bbox','image','mean_intensity']
    properties_ring = ['label','centroid','mean_intensity']

    data = viewer.layers['tracking'].data
    properties = viewer.layers['tracking'].properties

    # get the position in time
    myT = viewer.dims.current_step[0]

    # populate intensity image with data from a given frame
    intIm[:,:,0] = myIm[myT,:,:]

    for fr,x in zip(np.arange(1,len(myIm_signal_list)+1),myIm_signal_list):
        intIm[:,:,fr] = x[myT,:,:]

    # get my label
    myLabel = layer_inProgress.selected_label

    # create mask with only a selected object
    single_label_im = myLabels[myT,:,:].copy()
    single_label_im[single_label_im != myLabel]=0

    # find features of the new object
    cellData = pd.DataFrame(measure.regionprops_table(single_label_im, properties=regProps,intensity_image=intIm))

    # cut out the object as a small image
    x = int(cellData['centroid-0'])
    y = int(cellData['centroid-1'])
    myFrame = 100
    small_im = single_label_im[x-myFrame:x+myFrame,y-myFrame:y+myFrame]

    # change small image into a ring
    rings = fov_f.make_rings(small_im,width=6,gap=1)

    # put small rings image back into the whole frame
    single_label_im[x-myFrame:x+myFrame,y-myFrame:y+myFrame]=rings

    # measure properties of the ring
    rings_prop = measure.regionprops_table(single_label_im, properties=properties_ring,intensity_image=intIm)
    rings_prop = pd.DataFrame(rings_prop)

    # put nucleus and ring data together
    cellData = pd.merge(cellData,rings_prop,how='inner',on='label',suffixes=('_nuc', '_ring'))

    # modify names
    myNames = list(cellData.columns)
    myNames[2] = 'centroid-0'
    myNames[3] = 'centroid-1'
    cellData.columns = myNames

    # add aditional info
    cellData['t'] = myT
    cellData['track_id'] = cellData['label']

    # collect information about this label and this time point to calculate 
    info_track = cellDataAll.loc[:,['track_id','parent','root','generation','accepted']].drop_duplicates()
    info_frame = cellDataAll.loc[cellDataAll.t==myT,['t','background_01','background_02','background_03','background_04']].drop_duplicates()

    # merge it to the data of this frame
    cellData = cellData.merge(info_track,on='track_id')
    cellData = cellData.merge(info_frame,on='t')

    # calculate corrected signals
    for ch in np.arange(1,5):

        cellData[f'intensity_{str(ch).zfill(2)}_nuc_corr'] = cellData[f'mean_intensity-{ch-1}_nuc'] - cellData[f'background_{str(ch).zfill(2)}']
        cellData[f'intensity_{str(ch).zfill(2)}_ring_corr'] = cellData[f'mean_intensity-{ch-1}_ring'] - cellData[f'background_{str(ch).zfill(2)}']

    # add additional columns
    cellData['DHB_ratio'] = cellData[f'intensity_02_ring_corr']/cellData[f'intensity_02_nuc_corr']
    cellData['cyc_D_over_p21'] = cellData[f'intensity_03_nuc_corr']/cellData[f'intensity_04_nuc_corr']

    # swap in the general data frame
    cellDataAll.drop(cellDataAll[cellDataAll.t==myT].index,axis=0,inplace=True)
    cellDataAll = cellDataAll.append(cellData,ignore_index=True)


    # define objects to change 
    changeIndex = (data[:,1]==myT)

    # modify data of the track layer
    frameData = np.array(cellData.loc[:,['label','t','centroid-0','centroid-1']])

    data = np.delete(data,changeIndex,axis=0)

    data = np.vstack([data, frameData])

    # modify properties of the track layer

    cellData['state'] = 5

    for tProp in properties.keys():

        properties[tProp] = np.delete(properties[tProp],changeIndex)
        properties[tProp] = np.append(properties[tProp], cellData[tProp])

    # change tracks layer
    viewer.layers['tracking'].data = data
    viewer.layers['tracking'].properties = properties

    # change status
    viewer.status = f'Frame {myT} was modified.' 

Wall time: 770 ms


C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [348]:
# transfer modification of labels 
# -> into tracks layer
# -> into cellDataAll

# at the moment - triggered by 'u' key on labels layer

regProps = ['label', 'area','centroid','major_axis_length','minor_axis_length','bbox','image','mean_intensity']

@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    global cellDataAll
    
    data = viewer.layers['tracking'].data
    properties = viewer.layers['tracking'].properties

    # get the position
    myT = viewer.dims.current_step[0]

    # create intensity image
    intIm = np.moveaxis([myIm[myT,:,:],myIm_signal[myT,:,:],my_Labels[myT,:,:]],0,2)
    
    print('hello')

    # define objects to change 
    changeIndex = (data[:,1]==myT)

    # find new features of all the objects in the frame
    cellData = pd.DataFrame(measure.regionprops_table(inProgressTracks[myT,:,:], properties=regProps, intensity_image=intIm))
    cellData['t'] = myT
    cellData['track_id'] = cellData['mean_intensity-2'].astype(int) # measurement directly from the labels layer
    cellData.drop('mean_intensity-2',axis=1,inplace=True)

    # find info for labels if existing

    # create a unique data frame
    info_data = cellDataAll.loc[:,['track_id','parent','root','generation','accepted']].drop_duplicates()

    # merge it to the data of this frame
    cellData = cellData.merge(info_data,on='track_id')

    # swap in the general data frame
    cellDataAll.drop(cellDataAll[cellDataAll.t==myT].index,axis=0,inplace=True)
    cellDataAll = cellDataAll.append(cellData,ignore_index=True)

    # modify data of the track layer
    frameData = np.array(cellData.loc[:,['label','t','centroid-0','centroid-1']])

    data = np.delete(data,changeIndex,axis=0)

    data = np.vstack([data, frameData])

    # modify properties of the track layer

    cellData['state'] = 5

    for tProp in properties.keys():

        properties[tProp] = np.delete(properties[tProp],changeIndex)
        properties[tProp] = np.append(properties[tProp], cellData[tProp])

    # change tracks layer
    viewer.layers['tracking'].data = data
    viewer.layers['tracking'].properties = properties

    # change status
    viewer.status = f'Frame {myT} was modified.' 

    
# -> for changed traces re-generate small movie

## Stack viewer

In [16]:
def update_stack(viewer_stack):
    
    # check which object is selected
    try:
        myTrack = viewer_stack.layers['objects'].selected_label
    except KeyError:
        myTrack = 1
    
    # ask for an update
    stack_track,stack_labels,stack_signal_list = gallery_functions.stack_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display new layers
    my_napari.display_set(viewer_stack,stack_track,stack_labels,stack_signal_list,colors_list,names_list,label_contour=0)
    
def update_graph(viewer_stack,myTrack):
    
    global h
    
    viewer_stack.window.remove_dock_widget(h)
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myTrack) 
    h = viewer_stack.window.add_dock_widget(mpl_widget)

In [18]:
# initialize stack viewer
myTrack = 50 
viewer_stack = napari.Viewer()

update_stack(viewer_stack)

# init graph - there must be a more elegant solution (without this global handle)
mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myTrack) 
h = viewer_stack.window.add_dock_widget(mpl_widget)

# add an update button
# in the future - connect it to directly changing the selected layer
@magicgui(call_button='Update stack')
def button_stack(viewer_stack: Viewer):
    
    global h
    
    myTrack = viewer_stack.layers['objects'].selected_label
    
    # update stack
    update_stack(viewer_stack)
    
    # update graph
    update_graph(viewer_stack,myTrack)
    
viewer_stack.window.add_dock_widget(button_stack,area='bottom')

In [16]:
def update_gallery(myTrack):
    
    global viewer_gallery
    
    # ask for an update
    gallery_track,gallery_labels,gallery_signal_list = gallery_functions.gallery_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display an update
    try:
        viewer_gallery.close()
    except:
        pass

    viewer_gallery = napari.Viewer()
    
    my_napari.display_set(viewer_gallery,gallery_track,gallery_labels,gallery_signal_list,colors_list,names_list,label_contour=2)

In [100]:
# right click for accepting points
# toggles status of a track and visualizes it in the accepted objects layer if changed into accepted

@layer_inProgress.mouse_drag_callbacks.append
def accept_track(layer, event):

    if(event.button == 2):
        
        # look up cursor position
        x = int(viewer.cursor.position[1])
        y = int(viewer.cursor.position[2])
        
        # get data
        inProgressTracks = layer_inProgress.data
        
        # check which cell was clicked
        myTrackNum = inProgressTracks[viewer.dims.current_step[0],x,y]
        
        if myTrackNum > 0:

            # check status
            trackStatus = list(cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'])[0]

            # change status of this track
            cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'] = not(trackStatus)

            # regenerate accepted points
            selData=cellDataAll.loc[cellDataAll.accepted==True,:]
            acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

            # update viewer    
            layer_accepted.data = acceptedPoints

In [132]:
# swapping tracks based on 4 points

@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):

    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)


        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)


            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)


            # change graph
            children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])

            for child in children_1:

                graph[int(child)] = track_2

            children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

            for child in children_2:

                graph[int(child)] = track_1


            # change tracking data
            track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
            track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

            data[track2_change,0] = track_1
            data[track1_change,0] = track_2

            # change info about the parent 
            track1_change = ((properties['parent']==track_1) & (data[:,1]>=frame_change))
            track2_change = ((properties['parent']==track_2) & (data[:,1]>=frame_change))

            properties['parent'][track1_change] = track_2
            properties['parent'][track2_change] = track_1

            # change info about the root
            root1 = properties['root'][track1_change][0]
            root2 = properties['root'][track2_change][0]

            track1_change = ((properties['root']==root1) & (data[:,1]>=frame_change))
            track2_change = ((properties['root']==root2) & (data[:,1]>=frame_change))

            properties['root'][track1_change] = root2
            properties['root'][track2_change] = root1

            # change info about the generation
            gen1 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_1)]
            gen2 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_2)]

            properties['generation'][track1_change] = properties['generation'][track1_change] + gen2 - gen1
            properties['generation'][track2_change] = properties['generation'][track2_change] + gen1 - gen2



            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            viewer.layers['data'].graph = graph

            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [170]:
importlib.reload(gallery_functions)
importlib.reload(my_napari)

<module 'napari_display_functions' from 'D:\\BARC\\napari_tracking_manual\\napari_display_functions.py'>

In [157]:
myInd=582

In [167]:
np.min([x+int(imSize/2),myIm.shape[1]])

1817

In [169]:
imSize=100
x = int(data[myInd,2])
y = int(data[myInd,3])
myFrame = int(data[myInd,1])

# look up start frame
startFrame = int(np.min(data[data[:,0]==myTrack,1]))

t = myFrame - startFrame

# calculate how to cut
row_start = np.max([x-int(imSize/2),0])
row_stop = np.min([x+int(imSize/2),myIm.shape[1]])

column_start = np.max([y-int(imSize/2),0])
column_stop = np.min([y+int(imSize/2),myIm.shape[2]])


# calculate how to place
row_in_start = int((imSize - (row_stop-row_start))/2)
row_in_stop = int(row_in_start + (row_stop-row_start))

column_in_start = int((imSize - (column_stop-column_start))/2)
column_in_stop = int(row_in_start + (column_stop-column_start))
print(row_start)
print(row_stop)
print(column_start)
print(column_stop)
print(row_in_start)
print(row_in_stop)
print(column_in_start)
print(column_in_stop)

1717
1817
2578
2678
0
100
0
100


In [110]:
mpl_widget = my_napari.create_graph_widget(track_intensity,signal_intensity_ch,colors_list,names_list,cellDataAll,myTrack)

In [99]:
@magicgui(call_button='Update gallery')
def button_gallery(viewer: Viewer):
    
    myTrack = viewer.layers['objects'].selected_label
    update_gallery(myTrack)
    
    mpl_widget = my_napari.create_graph_widget(track_intensity,signal_intensity_ch,colors_list,names_list,cellDataAll,myTrack) 
    viewer_gallery.window.add_dock_widget(mpl_widget)

viewer.window.add_dock_widget(button_gallery,area='bottom')

In [68]:
@track_layer.bind_key('s',overwrite=True)
def update_inProgress(viewer):

    viewer_small = napari.Viewer()
    myTrack = 1 
    
    # create gallery
    
    viewer_small.add_image(myGallery,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

In [199]:
data[data[:,0]==1092,:]

array([[1092.        ,  318.        , 2162.92535971, 2026.29406475],
       [1092.        ,  319.        , 2162.92535971, 2026.29406475],
       [1092.        ,  320.        , 2139.88372093, 2058.0455814 ]])

In [201]:
data[(data[:,0]==7) & (data[:,1]==318),:]

array([[   7.        ,  318.        , 2172.5382482 , 2015.13455069]])

In [ ]:
# background correction for traces

In [ ]:
# cytoplasmic signal for selected traces

In [ ]:
# move graphs from matplotlib to qt and add them vertical lines
# add numbering of the frames to gallery

In [ ]:
# what to do with ghost objects

In [ ]:
# key binding for linking function

# check if parent have different offspring

# no other offspring
# connect points

# other offspring
# connect
# add to the graph

# what if objects didn;t